In [2]:
import torch
print(torch.__version__)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

2.2.0


'cpu'

定义参数

In [3]:
model_name="jbochi/madlad400-3b-mt"
source_lang="en"
target_lang="de"
dataset_name="wmt16"

下载读取WMT数据集

In [4]:
from datasets import load_dataset

# Download the WMT dataset
dataset = load_dataset(dataset_name, f"de-en")
translation=dataset["test"]["translation"][:100]
source_texts = [t[source_lang] for t in translation]  # Adjust the size as needed
reference_texts = [t[target_lang] for t in translation]
del dataset
del translation

Found cached dataset wmt16 (C:/Users/tiantian/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)


  0%|          | 0/3 [00:00<?, ?it/s]

测试模型

In [10]:
kwargs=dict(num_workers=0,batch_size=16,src_lang='en',
    tgt_lang='de',max_length=400)

In [6]:
from transformers import pipeline

translation_pipeline = pipeline(task=f"translation", model=model_name,device=device)
translation_pipeline('<2de> hello world', **kwargs)

[{'translation_text': 'Hallo Welt!'}]

生成翻译句子

In [11]:
import pandas as pd
translations = translation_pipeline([f'<2de> {st}' for st in source_texts],**kwargs)
translations = [translation['translation_text'] for translation in translations]
# 输出翻译结果
df = pd.DataFrame({
    'Source': source_texts,
    'Translation': translations
})
df

,Source,Translation
0,Obama receives Netanyahu,Obama empfängt Netanjahu
1,The relationship between Obama and Netanyahu i...,Die Beziehung zwischen Obama und Netanjahu ist...
2,The two wanted to talk about the implementatio...,Die beiden wollten über die Umsetzung des inte...
3,The meeting was also planned to cover the conf...,"Geplant war auch, den Konflikt mit den Palästi..."
4,Relations between Obama and Netanyahu have bee...,Die Beziehungen zwischen Obama und Netanjahu s...
...,...,...
95,Others have dismissed him as a joke.,Andere haben ihn als Witz abgetan.
96,And some are holding out for an implosion.,Und einige halten sich für eine Implosion bereit.
97,But no matter how some Republicans are trying ...,Aber egal wie sehr einige Republikaner versuch...
98,Ten of the last 11 national polls have shown D...,Zehn der letzten elf nationalen Umfragen haben...


使用sacrebleu评价模型

In [14]:
import sacrebleu
bleu_score = sacrebleu.corpus_bleu(translations, [reference_texts]).score
f"BLEU Score: {bleu_score}"

'BLEU Score: 48.32562066513895'

写入文件

In [13]:
with open("./BLEU_result.txt","a",encoding="utf-8") as f:
    f.write(f"\n{model_name} {source_lang} {target_lang} {bleu_score}")